In [1]:
#Import relevant libraries 
import pandas as pd
import altair as alt
import numpy as np
from ipywidgets import interact

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
#Import sunshine data from the Data folder
data = pd.read_csv('gdrive/My Drive/Grad School-Files/CSE 512/CSE 512: A3/Data/Modified_ Inflation_data2.csv')
data['Value'] = round(data['Value'], 2)
data['Inflation'] = round(data['Inflation'], 2)
data

,Year,Inflation,Measure,Value,Recession,Recession2,Recession_Type
0,1977,6.50,Unemployment Rate,7.05,0,0,
1,1977,6.50,Wage Change Rate,10.49,0,0,
2,1977,6.50,Mortage,8.84,0,0,
3,1977,6.50,WTI Crude Oil,13.06,0,0,
4,1978,7.63,Unemployment Rate,6.07,0,0,
...,...,...,...,...,...,...,...
175,2020,1.23,WTI Crude Oil,-31.16,80,-60,COVID-19 \nPandemic
176,2021,4.70,Unemployment Rate,5.36,0,0,
177,2021,4.70,Wage Change Rate,9.35,0,0,
178,2021,4.70,Mortage,2.96,0,0,


In [4]:
data['Year'] = pd.to_datetime(data['Year'], format='%Y')
data

,Year,Inflation,Measure,Value,Recession,Recession2,Recession_Type
0,1977-01-01,6.50,Unemployment Rate,7.05,0,0,
1,1977-01-01,6.50,Wage Change Rate,10.49,0,0,
2,1977-01-01,6.50,Mortage,8.84,0,0,
3,1977-01-01,6.50,WTI Crude Oil,13.06,0,0,
4,1978-01-01,7.63,Unemployment Rate,6.07,0,0,
...,...,...,...,...,...,...,...
175,2020-01-01,1.23,WTI Crude Oil,-31.16,80,-60,COVID-19 \nPandemic
176,2021-01-01,4.70,Unemployment Rate,5.36,0,0,
177,2021-01-01,4.70,Wage Change Rate,9.35,0,0,
178,2021-01-01,4.70,Mortage,2.96,0,0,


In [7]:
measurechart = alt.Chart(data).mark_line().encode(
    alt.X('Year:T'),
    alt.Y('Value:Q', axis = alt.Axis(title = 'Percentage %')),
    color = alt.Color('Measure:N')
)
inflationline = alt.Chart(data).mark_line().encode(
    alt.X('Year:T'),
    alt.Y('Inflation:Q'),
    color = alt.value('#5F6A6A')
)

alt.layer(measurechart, inflationline).interactive(bind_y = False)

alt.LayerChart(...)

In [8]:
measurements = ['Unemployment Rate','Wage Change Rate','Mortage','WTI Crude Oil']

selection = alt.selection_single(
    fields=['Measure'], # limit selection to the Measure field
    init={'Measure': 'Wage Change Rate'}, # start with unemployment rate
    bind=alt.binding_select(options= measurements, name='Select a Measure to compare against Inflation ') # bind to a menu of unique genre values
)

scales = alt.selection_interval(bind='scales', encodings=['x'])

measurelines = alt.Chart(data).mark_line().encode(
    alt.X('Year:T', axis = alt.Axis(title = 'Year')),
    alt.Y('Value:Q',  axis = alt.Axis(title = 'Percentage (%)')),
    color = alt.Color('Measure', scale = alt.Scale(
        domain =['Inflation', 'Unemployment Rate','Wage Change Rate','Mortage','WTI Crude Oil'],
        range= ['#5F6A6A', '#E74C3C ','#16A085','#B7950B ','#3498DB '])
    ),
    opacity = alt.condition(selection, alt.value(0.9),alt.value(0.05)),
)

label = alt.selection_single(
    encodings=['x'], # limit selection to x-axis value
    on='mouseover',  # select on mouseover events
    nearest=True,    # select data point nearest the cursor
    empty='none'     # empty selection includes no data points
)

rec = alt.Chart(data).mark_bar(color = '#dedede', size = 20).encode(
    alt.X('Year:T'),
    alt.Y('Recession:Q', axis = alt.Axis(title = 'Percentage (%)')),
    alt.Y2('Recession2:Q'),
)

line_chart = measurelines.add_selection(
    selection
)

measurementpoint_chart = alt.layer(
  
   alt.Chart().mark_rule(color='#aaa').encode(
        x='Year:T'
    ).transform_filter(label),
    
    # add circle marks for selected time points, hide unselected points
    measurelines.mark_circle().encode(
        opacity=alt.condition(label, alt.value(1), alt.value(0))
    ).add_selection(label),

    inflationline.mark_circle().encode(
        opacity=alt.condition(label, alt.value(1), alt.value(0))
    ).transform_filter(label),


    # add white stroked text to provide a legible background for labels
    measurelines.mark_text(align='left', dx=5, dy=-5).encode(
        text='Value:Q'
    ).transform_filter(label),

    rec.mark_text(align = 'center', dx = 5, dy = 20).encode(
        text = 'Recession_Type:N'
    ).transform_filter(label),

    inflationline.mark_text(align='left', dx=5, dy=-5).encode(
        text='Inflation:Q'
    ).transform_filter(label),

    data=data  
)

chart = alt.layer(
   rec,
   line_chart,
   inflationline,
  measurementpoint_chart
).encode(
    alt.Y(axis = alt.Axis(title = 'Percentage (%)')),
    alt.X(axis = alt.Axis(grid = False, minExtent = 50)),

).properties(
    width = 700,
    height = 500,
).transform_filter(
    selection
).interactive(bind_y = False)

chart



alt.LayerChart(...)

In [ ]:
chart.save('gdrive/My Drive/Grad School-Files/CSE 512/CSE 512: A3/chart_final.html')